In [10]:
import threading
import time
import inspect

class Thread(threading.Thread):
    def __init__(self, t, *args):
        threading.Thread.__init__(self, target=t, args=args)
        self.start()

class GlobalCounter(object):
    counter = 0  
    @classmethod
    def incre(cls):
        cls.counter += 1  
        if cls.counter % 100 == 0:
            print cls.counter    
        time.sleep(0.1)  

def incr_thread():
    for x in xrange(0, 100):
        GlobalCounter.incre()

def main():    
    my_threads = []
    for i in xrange(0, 10):
        my_thread = Thread(incr_thread)
        my_threads.append(my_thread)
    
    for my_thread in my_threads:
        my_thread.join()
    
    print GlobalCounter.counter
    
if __name__ == '__main__':
    main()

100
200
300
400
500
600
700
800
900
989


### Use Lock 

In [9]:
import threading
import time
import inspect

class Thread(threading.Thread):
    def __init__(self, t, *args):
        threading.Thread.__init__(self, target=t, args=args)
        self.start()

lock = threading.Lock()

class GlobalCounter(object):
    counter = 0 
    lock = threading.Lock()
    @classmethod
    def incre(cls):
        with GlobalCounter.lock:
            cls.counter += 1  
            if cls.counter % 100 == 0:
                print cls.counter
        time.sleep(0.1)  
        
def incr_thread():
    for x in xrange(0, 100):
        GlobalCounter.incre()

my_threads = []
for i in xrange(0, 10):
    my_thread = Thread(incr_thread)
    my_threads.append(my_thread)
    
for my_thread in my_threads:
    my_thread.join()
    
print GlobalCounter.counter

100
200
300
400
500
600
700
800
900
1000
1000


### Producer/Consumer with Condition object

In [16]:
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG, format='(%(threadName)-9s) %(message)s',)

resources = []
def consumer(cv, required_resources_num=1):
    global resources
    logging.debug('Consumer thread started ...')
    for i in xrange(0, required_resources_num):
        with cv:
            while not resources:
                logging.debug('Consumer waiting ...')
                cv.wait()
            logging.debug('Consumer consumed the resource')
            resources.pop()

def producer(cv, available_resources):
    global resources    
    logging.debug('Producer thread started ...')    
    for i in xrange(0, available_resources):
        with cv:
            logging.debug('Making resource available')
            logging.debug('Notifying to all consumers')
            resources.append(i)
            cv.notifyAll()
            time.sleep(1)

if __name__ == '__main__':
    condition = threading.Condition()
    cs1 = threading.Thread(name='consumer1', target=consumer, args=(condition, 5))
    cs2 = threading.Thread(name='consumer2', target=consumer, args=(condition,3))
    pd = threading.Thread(name='producer', target=producer, args=(condition,8))

    cs1.start()
    time.sleep(2)
    cs2.start()
    time.sleep(2)
    pd.start()

(consumer1) Consumer thread started ...
(consumer1) Consumer waiting ...
(consumer2) Consumer thread started ...
(consumer2) Consumer waiting ...
(producer ) Producer thread started ...
(producer ) Making resource available
(producer ) Notifying to all consumers
(consumer2) Consumer consumed the resource
(consumer1) Consumer waiting ...
(producer ) Making resource available
(producer ) Notifying to all consumers
(consumer2) Consumer consumed the resource
(consumer1) Consumer waiting ...
(producer ) Making resource available
(producer ) Notifying to all consumers
(consumer2) Consumer consumed the resource
(consumer1) Consumer waiting ...
(producer ) Making resource available
(producer ) Notifying to all consumers
(consumer1) Consumer consumed the resource
(producer ) Making resource available
(producer ) Notifying to all consumers
(consumer1) Consumer consumed the resource
(producer ) Making resource available
(producer ) Notifying to all consumers
(consumer1) Consumer consumed the reso